We'll start by defining some classes, let's create a flowline class to hold each flowline

In [1]:
import math
def Dist3D(A,B):
    d = math.sqrt((A[0]-B[0])*(A[0]-B[0])+\
                 (A[1]-B[1])*(A[1]-B[1])+\
                 (A[2]-B[2])*(A[2]-B[2]))
    return(d)
class FlowLine:
  def __init__(self):
    # we'll use negative values to indicate uninitalized quantities
    self.length = -1.0
    self.N = -1
    self.points = []
  def getN(self):
    self.N=len(self.points)
  def calcLength(self):
    self.length=0.0
    for i in range(len(self.points)-1):
        self.length = self.length + \
        Dist3D(self.points[i],self.points[i+1])
  def fracInterp(self,f):
    if (f<0.0):
        return(self.points[0])
    elif (f>1.0):
        return(self.points[self.N-1])
    else:
        n0=int(f*(self.N-1))
        x=f*(self.N-1)-n0
        val_x = self.points[n0][0] + x*(self.points[n0+1][0]-self.points[n0][0])
        val_y = self.points[n0][1] + x*(self.points[n0+1][1]-self.points[n0][1])
        val_z = self.points[n0][2] + x*(self.points[n0+1][2]-self.points[n0][2])
        return((val_x,val_y,val_z))


##Subset VLA
This routine filters out all but every *Nfrac*-th line of each segment of the .vla file. Note right now the code isn't smart enough to make sure that it includes the endpoint of each segment - should probably fix that.

In [1]:
    Nfrac = 40
    readfile=open('Shows\models\SDvision_Laniakea_streamlines_v001.vla',"r")
    writefile=open('Shows\models\Laniakea_streamlines_p025.vla',"w")
    count = 1
    for line in readfile:
        if (line.find('P ')==0):
            if (count !=1):
                writefile.write(oldline)                
                count=1;
        if (line.find('L ')!=0 or count%Nfrac ==0):
            writefile.write(line)
        if (line.find('L ')==0):
            count=count+1
            oldline=line
    writefile.close()
    readfile.close()

##Convert VLA
This routine converts the km/s units to Mpc in a .vla file.


In [110]:
from astropy import units as u
from astropy.cosmology import Planck13
readfile=open('Shows\models\Laniakea_streamlines_p025.vla',"r")
writefile=open('Shows\models\Laniakea_streamlines_Mpc_p025.vla',"w")
for line in readfile:
    if (line.find('L ')!=0 and line.find('P ')!=0):
        #pass along the header and blank lines
        writefile.write(line)
    else:
        elements = line.split()
        outline = elements[0]+ " %7.3f  %7.3f  %7.3f 1\n" % \
          (((float(elements[1])*u.km/u.s)/Planck13.H(0)).value, \
          ((float(elements[2])*u.km/u.s)/Planck13.H(0)).value, \
          ((float(elements[3])*u.km/u.s)/Planck13.H(0)).value )
        writefile.write(outline)
writefile.close()
readfile.close()

In [39]:
Planck13.H(0)

<Quantity 67.77 km / (Mpc s)>

##Creating Flow animation

###Import Flows

In [9]:
FlowList = []
readfile=open('Shows\models\SDvision_Laniakea_streamlines_v001.vla',"r")
#writefile=open('Shows\models\Laniakea_flow.speck',"w")
start=False
for line in readfile:
    if (line.find('P ')==0):
        if (start):
            FlowList.append(flowline)
        start=True
        elements = line.split()
        flowline = FlowLine()
        flowline.points.append((float(elements[1]),float(elements[2]),float(elements[3])))
    elif (line.find('L ')==0):
        elements = line.split()
        flowline.points.append((float(elements[1]),float(elements[2]),float(elements[3])))
FlowList.append(flowline)

for flow in FlowList:
    flow.getN()
    flow.calcLength()
    
readfile.close()

In [121]:
FlowList[3].fracInterp(0.5)

(-4598.8035, -1017.941, 4081.0985)

In [115]:
FlowList[3].points[500]

(-4604.142, -1011.665, 4079.613)

In [118]:
FlowList[3].points[499]

(-4593.465, -1024.217, 4082.584)

In [88]:
int(3.7)

3

In [15]:
from astropy import units as u
from astropy.cosmology import Planck13

startPositions = [0.0,0.2,0.4,0.6]
nTimesteps = 120
writefile=open('Shows\models\Laniakea_animation.speck',"w")
headerText="\
datavar 0 txnum\n\
texturevar txnum\n\
texture -a 1 star.tga\n\n"
writefile.write(headerText)

for i in range(nTimesteps):
    outline = "datatime %d\n" % i
    writefile.write(outline)
    time = i/(1.0*nTimesteps)
    for flow in FlowList:
        for startPos in startPositions:
            pos = flow.fracInterp(time+startPos)
            outline = "%7.3f  %7.3f  %7.3f 1\n" % \
              (((float(pos[0])*u.km/u.s)/Planck13.H(0)).value, \
              ((float(pos[1])*u.km/u.s)/Planck13.H(0)).value, \
              ((float(pos[2])*u.km/u.s)/Planck13.H(0)).value )
            writefile.write(outline)
writefile.close()


IndexError: list index out of range

In [14]:
writefile.close()